In [1]:
import os
from pydantic import BaseModel
from pydantic.fields import Field
from openai import OpenAI
from dotenv import load_dotenv
import prompts
import context_prompts
import yaml

In [2]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = OPENAI_API_KEY)

In [3]:
def answer(msg):
    response = client.chat.completions.create(
        
        model = "gpt-4o-mini",
        temperature = 0,
        messages=[
            {
                "role": "user",
                "content": msg
            }
        ]
    )
    return response.choices[0].message.content

In [4]:
def answer_RQ1(msg):
    response = client.chat.completions.create(
        
        model = "gpt-4o-mini",
        temperature = 0,
        messages=[
            {
        "role": "system",
        "content": "You are a data warehouse designer. I’m the end-user."
             },
            {
                "role": "user",
                "content": msg
            }
        ]
    )
    return response.choices[0].message.content

In [5]:
def save_to_file(filename, content):
    """
    Saves content to a file.
    """
    with open(filename, 'w') as file:
        file.write(content)

In [6]:
filename = "input/file.yml"
with open(filename, 'r') as file:
    content = file.read()

In [7]:
for q_prompt in dir(prompts):

    if q_prompt.startswith("RQ1"):
        prompt_value = getattr(prompts, q_prompt)
        answer_text = answer_RQ1(prompt_value)
        save_to_file(f"output/{q_prompt}.txt", answer_text)


    for context_prompt in dir(context_prompts):
        
        if context_prompt.startswith("__"):  # Skip built-in attributes
            continue

        if q_prompt.startswith("__"):  # Skip built-in attributes
            continue

        # Combine context and question prompts for RQ2
        if context_prompt.endswith("q2") and q_prompt.startswith("RQ2"):
            combined_prompt = f"{context_prompt.format(content=filename,msg=q_prompt)}\n{getattr(prompts, q_prompt)}"
            answer_text = answer(combined_prompt)
            var_name = f"{context_prompt}_{q_prompt}"
            save_to_file(f"output/{var_name}.yaml", answer_text)

        # Combine context and question prompts for RQ3
        elif context_prompt.endswith("q3") and q_prompt.startswith("RQ3"):
            combined_prompt = f"{context_prompt.format(content=filename,msg=q_prompt)}\n{getattr(prompts, q_prompt)}"
            answer_text = answer(combined_prompt)
            var_name = f"{context_prompt}_{q_prompt}"
            save_to_file(f"output/{var_name}.yaml", answer_text)